In [1]:
from langchain.llms import Cohere
from langchain import PromptTemplate, LLMChain

In [2]:
template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [3]:
llm = Cohere(cohere_api_key="xCRrVzZjuPM5HN6WFKM1eykBBwHMezrMhaQ0AaD7")

In [8]:
COHERE_API_KEY="xCRrVzZjuPM5HN6WFKM1eykBBwHMezrMhaQ0AaD7"

In [4]:
llm_chain = LLMChain(prompt=prompt, llm=llm)

In [5]:
question = "What NFL team won the Super Bowl in the year Justin Beiber was born?"

llm_chain.run(question)

'\nJustin Bieber was born on March 1, 1994.  The Super Bowl is the annual American football game that determines the league champion.  The Super Bowl has been played each year since 1967.  The Super Bowl is typically played in February.  The 1994 Super Bowl was played on January 30, 1994.  The Super Bowl is played by two teams.  The two teams in the 1994 Super Bowl were the San Francisco 49ers and the San Diego Chargers.  The San Francisco 49ers won the 1994 Super Bowl.  The San Francisco 49ers were the NFL team that won the Super Bowl in the year Justin Bieber was born.'

In [15]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory, ReadOnlySharedMemory
from langchain import OpenAI, LLMChain, PromptTemplate
import os
from langchain import SerpAPIWrapper

In [17]:
## This is a bad practice for using the memory.
## Use the ReadOnlySharedMemory class, as shown above.

template = """This is a conversation between a human and a bot:

{chat_history}

Write a summary of the conversation for {input}:
"""
GOOGLE_API_KEY="8e9fe6452300e05fd6301b16a2b04cf69219a41bb8195df3496db2605d97a72a"
prompt = PromptTemplate(input_variables=["input", "chat_history"], template=template)
memory = ConversationBufferMemory(memory_key="chat_history")
summry_chain = LLMChain(
    llm=Cohere(cohere_api_key="xCRrVzZjuPM5HN6WFKM1eykBBwHMezrMhaQ0AaD7"),
    prompt=prompt,
    verbose=True,
    memory=memory,  # <--- this is the only change
)
os.environ["SERPAPI_API_KEY"] = "8e9fe6452300e05fd6301b16a2b04cf69219a41bb8195df3496db2605d97a72a"
search = SerpAPIWrapper()
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="Summary",
        func=summry_chain.run,
        description="useful for when you summarize a conversation. The input to this tool should be a string, representing who will read this summary.",
    ),
]

prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"],
)

llm_chain = LLMChain(llm=Cohere(temperature=0, cohere_api_key="xCRrVzZjuPM5HN6WFKM1eykBBwHMezrMhaQ0AaD7"), prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, memory=memory
)

In [18]:
agent_chain.run(input="What is ChatGPT?")



> Entering new AgentExecutor chain...
Thought: I need to answer the question: What is ChatGPT?
Action: Search
Action Input: ChatGPT
Observation: ChatGPT is an artificial intelligence chatbot developed by OpenAI and launched on November 30, 2022. It is notable for enabling users to refine and steer a conversation towards a desired length, format, style, level of detail, and language used.
Thought: I now know the answer is ChatGPT is an artificial intelligence chatbot developed by OpenAI and launched on November 30, 2022. It is notable for enabling users to refine and steer a conversation towards a desired length, format, style, level of detail, and language used.
Final Answer: ChatGPT is an artificial intelligence chatbot developed by OpenAI and launched on November 30, 2022. It is notable for enabling users to refine and steer a conversation towards a desired length, format, style, level of detail, and language used.

> Finished chain.


'ChatGPT is an artificial intelligence chatbot developed by OpenAI and launched on November 30, 2022. It is notable for enabling users to refine and steer a conversation towards a desired length, format, style, level of detail, and language used.'

In [23]:
os.environ["ZAPIER_NLA_API_KEY"] = "sk-ak-Bd0lcsupVm55rXykjmGK8hUpra"

from langchain.llms import OpenAI
from langchain.agents import initialize_agent
from langchain.agents.agent_toolkits import ZapierToolkit
from langchain.agents import AgentType
from langchain.utilities.zapier import ZapierNLAWrapper

In [24]:
zapier = ZapierNLAWrapper()
toolkit = ZapierToolkit.from_zapier_nla_wrapper(zapier)
agent = initialize_agent(
    toolkit.get_tools(), llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [27]:
agent.run(
    "what was the last message I received on discord by User with Username=Omni"
)



> Entering new AgentExecutor chain...
 I should find the user with the username Omni.
Action: Discord: Find User.
Action Input: ['Username']
Observation: Discord: Find User. is not a valid tool, try another one.
Thought:

RetryError: HTTPSConnectionPool(host='api.cohere.ai', port=443): Max retries exceeded with url: /generate (Caused by ResponseError('too many 429 error responses'))

In [ ]:
import requests

CHUNK_SIZE = 1024
url = "https://api.elevenlabs.io/v1/text-to-speech/<voice-id>"

headers = {
  "Accept": "audio/mpeg",
  "Content-Type": "application/json",
  "xi-api-key": "<xi-api-key>"
}

data = {
  "text": "Hi! My name is Bella, nice to meet you!",
  "model_id": "eleven_monolingual_v1",
  "voice_settings": {
    "stability": 0.5,
    "similarity_boost": 0.5
  }
}

response = requests.post(url, json=data, headers=headers)
with open('output.mp3', 'wb') as f:
    for chunk in response.iter_content(chunk_size=CHUNK_SIZE):
        if chunk:
            f.write(chunk)


In [58]:
from cohere import Client
co = Client(api_key="xCRrVzZjuPM5HN6WFKM1eykBBwHMezrMhaQ0AaD7")
res = co.Summarize(
    text=data[0].page_content,
    model="summarize-xlarge",
    length="long",
    temperature=0.3,
    additional_command="focusing on making a prompt for a LLM")
print(res.summary)

AttributeError: 'Client' object has no attribute 'Summarize'

In [57]:
import cohere
co = cohere.Client('xCRrVzZjuPM5HN6WFKM1eykBBwHMezrMhaQ0AaD7')

text=(
  "Ice cream is a sweetened frozen food typically eaten as a snack or dessert. "
  "It may be made from milk or cream and is flavoured with a sweetener, "
  "either sugar or an alternative, and a spice, such as cocoa or vanilla, "
  "or with fruit such as strawberries or peaches. "
  "It can also be made by whisking a flavored cream base and liquid nitrogen together. "
  "Food coloring is sometimes added, in addition to stabilizers. "
  "The mixture is cooled below the freezing point of water and stirred to incorporate air spaces "
  "and to prevent detectable ice crystals from forming. The result is a smooth, "
  "semi-solid foam that is solid at very low temperatures (below 2 °C or 35 °F). "
  "It becomes more malleable as its temperature increases.\n\n"
  "The meaning of the name \"ice cream\" varies from one country to another. "
  "In some countries, such as the United States, \"ice cream\" applies only to a specific variety, "
  "and most governments regulate the commercial use of the various terms according to the "
  "relative quantities of the main ingredients, notably the amount of cream. "
  "Products that do not meet the criteria to be called ice cream are sometimes labelled "
  "\"frozen dairy dessert\" instead. In other countries, such as Italy and Argentina, "
  "one word is used fo\r all variants. Analogues made from dairy alternatives, "
  "such as goat's or sheep's milk, or milk substitutes "
  "(e.g., soy, cashew, coconut, almond milk or tofu), are available for those who are "
  "lactose intolerant, allergic to dairy protein or vegan."
)

response = co.summarize(
  text=text,
)
print(response)

AttributeError: 'Client' object has no attribute 'summarize'

In [47]:
data = loader.load()
data[0]

Document(page_content='Introduction\nLangChain is a framework for developing applications powered by language models. It enables applications that are:\nData-aware: connect a language model to other sources of data\nAgentic: allow a language model to interact with its environment\nThe main value props of LangChain are:\nComponents: abstractions for working with language models, along with a collection of implementations for each abstraction. Components are modular and easy-to-use, whether you are using the rest of the LangChain framework or not\nOff-the-shelf chains: a structured assembly of components for accomplishing specific higher-level tasks\nOff-the-shelf chains make it easy to get started. For more complex applications and nuanced use-cases, components make it easy to customize existing chains or build new ones.\nGet started\u200b\nHere’s how to install LangChain, set up your environment, and start building.\nWe recommend following our Quickstart guide to familiarize yourself w